In [3]:
import json
import os
import sys
import pickle
import re

import numpy as np
import pandas as pd
sys.path.append('../..')
from networks import DualNet, DualNetEndToEnd, PrimalNet, PrimalNetEndToEnd, PrimalNetEndToEnd
import torch

def evaluate(data, primal_net, test_indices):        
    X = data.X[test_indices]
    Y_target = data.opt_targets["y_operational"][test_indices]
    
    # Forward pass through networks
    Y = primal_net(X)

    ineq_dist = data.ineq_dist(X, Y)
    eq_resid = data.eq_resid(X, Y)

    relative_ineq_dist = data.relative_ineq_dist(X, Y)
    relative_eq_resid = data.relative_eq_resid(X, Y)

    # Convert lists to arrays for easier handling
    obj_values = data.obj_fn(X, Y).detach().numpy()
    ineq_max_vals = torch.max(ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(ineq_dist, dim=1).detach().numpy()
    eq_max_vals = torch.max(torch.abs(eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    eq_mean_vals = torch.mean(torch.abs(eq_resid), dim=1).detach().numpy()

    relative_ineq_max_vals = torch.max(relative_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    relative_ineq_mean_vals = torch.mean(relative_ineq_dist, dim=1).detach().numpy()
    relative_eq_max_vals = torch.max(torch.abs(relative_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    relative_eq_mean_vals = torch.mean(torch.abs(relative_eq_resid), dim=1).detach().numpy()

    known_obj = data.obj_fn(X, Y_target).detach().numpy()
    # obj_values is negative
    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(relative_ineq_max_vals), np.mean(ineq_mean_vals), np.mean(relative_ineq_mean_vals), np.mean(eq_max_vals), np.mean(relative_eq_max_vals), np.mean(eq_mean_vals), np.mean(relative_eq_mean_vals)

def dual_evaluate(data, dual_net, test_indices):
    X = data.X[test_indices]
    # target_mu = data.mu[test_indices]
    # target_lamb = data.lamb[test_indices]
    target_mu = data.opt_targets["mu_operational"][test_indices]  
    target_lamb = data.opt_targets["lamb_operational"][test_indices]
    print(X.dtype)
    print(dual_net.net[0].weight.dtype)

    # Forward pass through networks
    mu, lamb = dual_net(X)

    obj_values = data.dual_obj_fn(X, mu, lamb).detach().numpy()
    known_obj = data.dual_obj_fn(X, target_mu, target_lamb).detach().numpy()
    # dual_ineq_dist = data.dual_ineq_dist(mu, lamb)
    dual_ineq_resid = data.dual_ineq_resid(mu, lamb)
    dual_ineq_dist = torch.clamp(dual_ineq_resid, 0)
    dual_eq_resid = data.dual_eq_resid(mu, lamb)

    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    ineq_max_vals = torch.max(dual_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    eq_max_vals = torch.max(torch.abs(dual_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(dual_ineq_dist, dim=1).detach().numpy()
    eq_mean_vals = torch.mean(torch.abs(dual_eq_resid), dim=1).detach().numpy()

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(ineq_mean_vals), np.mean(eq_max_vals), np.mean(eq_mean_vals)

repeats = 5
stats_dict = {}

# args = json.load(open('config.json'))
data_path = f"experiment-output/ch6/ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl"
data = pickle.load(open(data_path, 'rb'))

indices = torch.arange(data.X.shape[0])

for run, (path, name) in enumerate(zip(["experiment-output/ch5/plain-PDL/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746265816-296861",
             "experiment-output/ch5/repair-1/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746429988-424426",
             "experiment-output/ch5/repair-2/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746434902-974866"],
            ["plain", "repair1", "repair2"])):
    stats_dict[name] = {"predicted_obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": [], "relative_ineq_max": [], "relative_ineq_mean": [], "relative_eq_max": [], "relative_eq_mean": []}
    with open(os.path.join(path, 'args.json'), 'r') as f:
        args = json.load(f)
    # Compute sizes for each set
    train_size = int(args["train"] * data.X.shape[0])
    valid_size = int(args["valid"] * data.X.shape[0])
    # print(f"Train size: {train_size}, Valid size: {valid_size}, Test size: {data.X.shape[0] - train_size - valid_size}")


    # Split the indices
    train_indices = indices[:train_size]
    valid_indices = indices[train_size:train_size+valid_size]
    test_indices = indices[train_size+valid_size:]

    for i, repeat in enumerate(range(repeats)):
        
        directory = os.path.join(path, f"repeat:{repeat}")
        # directory = f"experiment-output/ch5-reproduction-nonconvex/{experiment}/repeat:{repeat}"
        # dual_net = DualNet(args, data=data)
        # dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        primal_net = PrimalNetEndToEnd2(args, data=data)
        primal_net.load_state_dict(torch.load(os.path.join(directory, 'primal_weights.pth'), weights_only=True))
        # obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate(data, dual_net, test_indices)
        obj_val, known_obj, opt_gap, ineq_max, relative_ineq_max, ineq_mean, relative_ineq_mean, eq_max, relative_eq_max, eq_mean, relative_eq_mean = evaluate(data, primal_net, test_indices)
        stats_dict[name]["predicted_obj"].append(obj_val)
        stats_dict[name]["known_obj"].append(known_obj)
        stats_dict[name]["opt_gap"].append(opt_gap)
        stats_dict[name]["ineq_max"].append(ineq_max)
        stats_dict[name]["ineq_mean"].append(ineq_mean)
        stats_dict[name]["eq_max"].append(eq_max)
        stats_dict[name]["eq_mean"].append(eq_mean)
        stats_dict[name]["relative_ineq_max"].append(relative_ineq_max)
        stats_dict[name]["relative_ineq_mean"].append(relative_ineq_mean)
        stats_dict[name]["relative_eq_max"].append(relative_eq_max)
        stats_dict[name]["relative_eq_mean"].append(relative_eq_mean)


stats_dict

        

FileNotFoundError: [Errno 2] No such file or directory: 'experiment-output/ch6/ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl'

In [13]:
# Prepare separate dictionaries
objective_summary = {"Experiment": [], "Optimal Obj": [], "Predicted Obj": [], "OptGap (%)": []}
regular_summary = {"Experiment": [], "IneqMax": [], "IneqMean": [], "EqMax": [], "EqMean": []}
relative_summary = {"Experiment": [], "Relative IneqMax": [], "Relative IneqMean": [], "Relative EqMax": [], "Relative EqMean": []}

for experiment, metrics in stats_dict.items():
    objective_summary["Experiment"].append(experiment)
    objective_summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3f}")
    objective_summary["Predicted Obj"].append(f"{np.mean(metrics['predicted_obj']):.3f}({np.std(metrics['predicted_obj']):.3f})")
    objective_summary["OptGap (%)"].append(f"{np.mean(metrics['opt_gap']):.3f}({np.std(metrics['opt_gap']):.3f})")

    regular_summary["Experiment"].append(experiment)
    regular_summary["IneqMax"].append(f"{np.mean(metrics['ineq_max']):.3f}({np.std(metrics['ineq_max']):.3f})")
    regular_summary["IneqMean"].append(f"{np.mean(metrics['ineq_mean']):.3f}({np.std(metrics['ineq_mean']):.3f})")
    regular_summary["EqMax"].append(f"{np.mean(metrics['eq_max']):.3f}({np.std(metrics['eq_max']):.3f})")
    regular_summary["EqMean"].append(f"{np.mean(metrics['eq_mean']):.3f}({np.std(metrics['eq_mean']):.3f})")

    relative_summary["Experiment"].append(experiment)
    relative_summary["Relative IneqMax"].append(f"{np.mean(metrics['relative_ineq_max']):.3f}({np.std(metrics['relative_ineq_max']):.3f})")
    relative_summary["Relative IneqMean"].append(f"{np.mean(metrics['relative_ineq_mean']):.3f}({np.std(metrics['relative_ineq_mean']):.3f})")
    relative_summary["Relative EqMax"].append(f"{np.mean(metrics['relative_eq_max']):.3f}({np.std(metrics['relative_eq_max']):.3f})")
    relative_summary["Relative EqMean"].append(f"{np.mean(metrics['relative_eq_mean']):.3f}({np.std(metrics['relative_eq_mean']):.3f})")

# Convert to DataFrames
df_obj = pd.DataFrame(objective_summary)
df_regular = pd.DataFrame(regular_summary)
df_relative = pd.DataFrame(relative_summary)

# Export LaTeX tables
latex_obj = df_obj.to_latex(index=False, escape=False)
latex_regular = df_regular.to_latex(index=False, escape=False)
latex_relative = df_relative.to_latex(index=False, escape=False)

# Print or write to file
print("% Objective Table")
print(latex_obj)
print("\n% Regular Constraint Table")
print(latex_regular)
print("\n% Relative Constraint Table")
print(latex_relative)

% Objective Table
\begin{tabular}{llll}
\toprule
Experiment & Optimal Obj & Predicted Obj & OptGap (%) \\
\midrule
plain & 29293.887 & 582586.024(2784.077) & 27271.364(168.671) \\
repair1 & 29293.887 & 80987.094(3916.860) & 2567.187(204.781) \\
repair2 & 29293.887 & 29310.046(5.399) & 0.705(0.209) \\
\bottomrule
\end{tabular}


% Regular Constraint Table
\begin{tabular}{lllll}
\toprule
Experiment & IneqMax & IneqMean & EqMax & EqMean \\
\midrule
plain & 711.543(14.045) & 46.839(1.263) & 5267.664(13.714) & 3261.341(7.769) \\
repair1 & 34.633(10.675) & 1.533(0.459) & 0.000(0.000) & 0.000(0.000) \\
repair2 & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) \\
\bottomrule
\end{tabular}


% Relative Constraint Table
\begin{tabular}{lllll}
\toprule
Experiment & Relative IneqMax & Relative IneqMean & Relative EqMax & Relative EqMean \\
\midrule
plain & 166.923(9.203) & 10.407(0.553) & 0.169(0.000) & 0.136(0.000) \\
repair1 & 0.002(0.000) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) \

In [4]:
# Get the quality of dual solutions:
dual_stats_dict = {}
for experiment in ["simple"]:
    dual_stats_dict[experiment] = {"obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    for repeat in range(repeats):
        directory = f"experiment-output/ch4/ch4-reproduction/{experiment}/repeat:{repeat}"
        data_path = f"data/QP_data/QP_type:{experiment}_var:100_ineq:50_eq:50_num_samples:10000.pkl"

        args = json.load(open('config.json'))
        data = pickle.load(open(data_path, 'rb'))
        dual_net = DualNet(args, data=data)
        dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate(data, dual_net, data.test_indices)
        dual_stats_dict[experiment]["obj"].append(obj_val)
        dual_stats_dict[experiment]["known_obj"].append(known_obj)
        dual_stats_dict[experiment]["opt_gap"].append(opt_gap)
        dual_stats_dict[experiment]["ineq_max"].append(ineq_max)
        dual_stats_dict[experiment]["ineq_mean"].append(ineq_mean)
        dual_stats_dict[experiment]["eq_max"].append(eq_max)
        dual_stats_dict[experiment]["eq_mean"].append(eq_mean)

dual_stats_dict

NameError: name 'repeats' is not defined

In [44]:
# Prepare final summary for LaTeX export
summary = {
    "Optimal Obj": [],
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}

for experiment, metrics in dual_stats_dict.items():
    summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3f}")
    summary["Predicted Obj"].append(f"{np.mean(metrics['obj']):.3e}({np.std(metrics['obj']):.3e})")
    summary["OptGap (\%)"].append(f"{np.mean(metrics['opt_gap']):.3e}({np.std(metrics['opt_gap']):.3e})")
    summary["IneqMax"].append(
        f"{np.mean(metrics['ineq_max']):.3f}({np.std(metrics['ineq_max']):.3f})"
    )
    summary["IneqMean"].append(
        f"{np.mean(metrics['ineq_mean']):.3f}({np.std(metrics['ineq_mean']):.3f})"
    )
    summary["EqMax"].append(
        f"{np.mean(metrics['eq_max']):.3f}({np.std(metrics['eq_max']):.3f})"
    )
    summary["EqMean"].append(
        f"{np.mean(metrics['eq_mean']):.3f}({np.std(metrics['eq_mean']):.3f})"
    )

df = pd.DataFrame(summary)

# For LaTeX export
latex_table = df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Optimal Obj & Predicted Obj & OptGap (\%) & IneqMax & IneqMean & EqMax & EqMean \\
\midrule
-15.037 & -7.778e+03(6.240e+03) & -5.175e+04(4.162e+04) & 0.023(0.006) & 0.003(0.001) & 0.000(0.000) & 0.000(0.000) \\
\bottomrule
\end{tabular}



In [5]:
import pandas as pd

# Mapping of original metric keys to display names
metric_names = {
    'predicted_obj': 'Predicted',
    'known_obj': 'Optimal',
    'opt_gap': 'Gap (\\%)',
    'ineq_max': 'Ineq. Max',
    'ineq_mean': 'Ineq. Mean',
    'eq_max': 'Eq. Max',
    'eq_mean': 'Eq. Mean'
}

metrics = list(metric_names.keys())
problem_types = list(stats_dict.keys())
repeats = range(len(stats_dict[problem_types[0]][metrics[0]]))

# Prepare multi-indexed rows: (Repeat, Renamed Metric)
rows = []
for r in repeats:
    for m in metrics:
        rows.append((f"Repeat {r}", metric_names[m]))

# Create DataFrame with multi-index rows and problem type columns
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(rows, names=["Repeat", "Metric"]),
                  columns=problem_types)

# Fill in the values
for problem in problem_types:
    for r in repeats:
        for m in metrics:
            df.loc[(f"Repeat {r}", metric_names[m]), problem] = stats_dict[problem][m][r]

# Convert to LaTeX with booktabs
latex_table = df.to_latex(
    escape=False,
    multirow=True,
    column_format='ll' + 'c' * len(problem_types),
    bold_rows=False,
    index_names=True,
    header=True,
    float_format="%.3f",
    caption="Detailed evaluation metrics per repeat and problem type.",
    label="tab:detailed-results",
    longtable=False,
    na_rep=''
)

print(latex_table)


NameError: name 'stats_dict' is not defined

In [6]:
!ls

ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl
plain-PDL
repair-1
repair-2
solution_analysis.ipynb
tables.ipynb
tensorboard_plots.ipynb


In [7]:
data_path = f"ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl"
data = pickle.load(open(data_path, 'rb'))

In [26]:
print(data.X.shape)
print(data.T)
print(data.opt_targets['obj'])
print(f"Number of samples: {data.n_samples}")
print(f"Per unit investment {data.pUnitInvestment}")

torch.Size([32768, 9])
range(1, 8761)
tensor([3155.5285, 1369.2450, 1467.8731,  ..., 2429.6445, 1942.3357,
        2349.2205], dtype=torch.float64)
Number of samples: 32768
Per unit investment tensor([[693.8206,  29.0175,  93.0703, 502.7595, 408.4569, 367.3533],
        [717.5849,  59.6154, 343.7348, 293.3847, 534.0417, 448.8720],
        [999.5382, 787.4132, 431.7997, 422.8027, 380.7295,   4.2450],
        ...,
        [877.2337, 294.3558, 843.5970, 354.7130, 681.9613,  15.1868],
        [101.2786, 799.3239,  11.1882, 245.6552, 825.4401, 470.7783],
        [762.1953, 645.9160, 338.5291, 656.8327, 953.4864, 705.0021]],
       dtype=torch.float64)
